In [26]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import os
import zipfile
import shutil
import multiprocessing

file_name=r'C:\Users\s3309\Desktop\Investment\html_files\tifrs-fr1-m1-ci-cr-2330-2022Q2.html'

def html_preprocess(file_name):
    with open(file_name, encoding="UTF-8") as f:
        data=f.read()
    soup = BeautifulSoup(data,'html.parser')

    data_all=[[] for i in range(5)]

    for sht in range(3):
        sheet=soup.find_all('table')[sht]
        cate=sheet.find_all('tr')[0].find_all('th')[0].find(class_='en').text.strip()

        i=1
        while i< len(sheet.find_all('tr')):
            if i==1:
                time_window=[]
                for time in sheet.find_all('tr')[i].find_all('th')[2:]:
                    time_window.append(time.find(class_='en').text)

            elif sheet.find_all('tr')[i].find_all('td')[0].text=='' and re.search(r'\d',sheet.find_all('tr')[i+1].find_all('td')[0].text):
                sub_cate=sheet.find_all('tr')[i].find_all('td')[1].find(class_='en').text.strip()

            elif re.search(r'\d',sheet.find_all('tr')[i].find_all('td')[0].text):

                for j,n in enumerate(sheet.find_all('tr')[i].find_all('td')[2:]):
                    data_all[0].append(cate)
                    data_all[1].append(sub_cate)
                    data_all[2].append(sheet.find_all('tr')[i].find_all('td')[1].find(class_='en').text.strip())
                    data_all[3].append(time_window[j])
                    data_all[4].append(n.text)
            i+=1

    table=pd.DataFrame()
    for i,col in enumerate(['Category','Sub-Category','Subject','Period','$']):
        table[col]=data_all[i]

    return table

def xml_preprocess(file_name):
    xml_df=pd.read_xml(file_name)
    xml_df.dropna(subset=['unitRef'],inplace=True)
    ui=xml_df.columns.to_list().index('unitRef')
    target_col=xml_df.columns[ui+1:]
    xml_df['Subject']=pd.NA
    xml_df['$']=pd.NA

    for col in target_col:
        xml_df.loc[~xml_df[col].isna(),'Subject']=col
        xml_df['$'].fillna(xml_df[col],inplace=True)
        xml_df.drop(col,axis=1,inplace=True)

    xml_df=xml_df[['contextRef','Subject','$']]
    xml_df['contextRef']=xml_df['contextRef'].replace(r'From','',regex=True)

    return xml_df


In [22]:
#unzip file
import os
import zipfile

for root, dirs, files in os.walk(r"C:\Users\s3309\Desktop\Investment\todo"):
    for file in files:
        if file.endswith('zip'):
            with zipfile.ZipFile(os.path.join(root,file), 'r') as zip_ref:
                zip_ref.extractall(r'C:\Users\s3309\Desktop\Investment\html_files')
                zip_ref.close()

In [29]:
xml_df=pd.DataFrame()
html_df=pd.DataFrame()

for root, dirs, files in os.walk(r"C:\Users\s3309\Desktop\Investment\todo"):
    for f in files[:30]:
        filename=f.split('.')[0]
        filename=filename.split('-')
        comp=filename[-2]
        qtr=filename[-1]
        abspath=os.path.join(root,f)

        if f.endswith('.xml'):
            temp=xml_preprocess(abspath)
            temp['Comp']=comp
            temp['Qtr']=qtr
            xml_df=pd.concat([xml_df,temp])
        else:
            temp=html_preprocess(abspath)
            temp['Comp']=comp
            temp['Qtr']=qtr
            html_df=pd.concat([html_df,temp])         
            


In [30]:
html_df['$']=html_df['$'].replace(r'^\(','-',regex=True)
html_df['$']=html_df['$'].replace(r',|\)','',regex=True).astype(float)
html_df['$']=html_df['$'].apply(lambda x:x*1000)

In [31]:
html_df['Mapping Col']=html_df['Subject'].apply(lambda x:re.sub(' ','',x).lower())
subcate_d={subject:subcate for subject,subcate in zip(html_df['Mapping Col'],html_df['Sub-Category'])}

In [32]:
xml_df['Mapping Col']=xml_df['Subject'].apply(lambda x:x.lower())
xml_df['Sub-Category']=xml_df['Mapping Col'].map(subcate_d)

In [34]:
import sqlite3

conn=sqlite3.connect('test')
xml_df.to_sql('XML_DF',conn)

26851

In [7]:
import test
from itertools import repeat
import pandas as pd
import multiprocessing

xml_df=pd.DataFrame()
html_df=pd.DataFrame()
for root, dirs, files in os.walk(r"C:\Users\s3309\Desktop\Investment\html_files"):
    file_list=files

with multiprocessing.Pool(8) as p:
    result=p.map(test.file_process,file_list[:30])

for i,n in enumerate(result):

    if 'Sub-Category' in n.columns:
        html_df=pd.concat([html_df,n])
    else:
        xml_df=pd.concat([xml_df,n])

import sqlite3

conn=sqlite3.connect('test')
html_df.to_sql('HTML_DF',conn)
xml_df.to_sql('XML_DF',conn)



In [17]:
float('-100')

-100.0

In [6]:
help(multiprocessing.Pool)

Help on method Pool in module multiprocessing.context:

Pool(processes=None, initializer=None, initargs=(), maxtasksperchild=None) method of multiprocessing.context.DefaultContext instance
    Returns a process pool object



In [41]:
xml_comp=[x for x in xml_df['Comp'].unique() if not x in xml_df2['Comp'].unique()]
html_comp=[x for x in html_df['Comp'].unique() if not x in html_df2['Comp'].unique()]






In [1]:
from multiprocessing import Pool

def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))

: 

: 

In [ ]:
'tifrs-fr0-m2-ci-ir-6032-2022Q2.html'
'tifrs-fr0-m2-ci-ir-6033-2022Q2.html


In [6]:
import multiprocessing  
import random
from multiprocessing import Pool
import defs

if __name__ == '__main__':
    pool = Pool()
    to_factor = [ random.randint(100000, 50000000) for i in range(20)]
    results = pool.map(defs.prime_factor, to_factor)
    for value, factors in zip(to_factor, results):
        print("The factors of {} are {}".format(value, factors))

The factors of 14326878 are [7163439, 2387813, 67, 35639, 157, 227]
The factors of 45512257 are [45512257, 7, 6501751, 37, 175723]
The factors of 34349663 are [34349663, 19, 1807877, 499, 3623]
The factors of 28136668 are [7034167, 7, 1004881, 83, 12107]
The factors of 26872128 are [419877, 139959, 46653, 15551]
The factors of 49737876 are [12434469, 4144823, 103, 40241]
The factors of 15761876 are [3940469, 13, 303113, 41, 7393]
The factors of 4998695 are [4998695, 5, 999739, 13, 76903, 53, 1451]
The factors of 14457694 are [7228847, 89, 81223]
The factors of 41446658 are [20723329, 11, 1883939]
The factors of 31564669 are [31564669]
The factors of 5493766 are [2746883, 43, 63881, 127, 503]
The factors of 24691622 are [12345811]
The factors of 30767224 are [3845903]
The factors of 2827227 are [2827227, 942409, 13, 72493]
The factors of 13199849 are [13199849, 13, 1015373, 193, 5261]
The factors of 44158000 are [2759875, 5, 551975, 5, 110395, 5, 22079]
The factors of 46646171 are [4664

In [4]:
driver_func()

Here we start


AttributeError: module 'test' has no attribute 'double'

In [49]:
logging.getLogger().setLevel(logging.INFO)